In [1]:
import os
import warnings

import matplotlib.pyplot as plt
import pandas as pd
from tqdm.auto import tqdm

warnings.filterwarnings("ignore")

C:\Users\saoedoss\AppData\Roaming\Python\Python310\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from darts import TimeSeries
from darts.dataprocessing.transformers import (
    InvertibleMapper,
    Mapper,
    MissingValuesFiller,
    Scaler,
)
from darts.models import (
    ARIMA,
    FFT,
    AutoARIMA,
    ExponentialSmoothing,
    NaiveDrift,
    NaiveMean,
    NaiveMovingAverage,
    NaiveSeasonal,
    Theta,
)
from darts.utils.statistics import check_seasonality
from darts.utils.utils import ModelMode, SeasonalityMode

In [3]:
import sys

# Adicionar a pasta 'src' ao caminho do sistema
sys.path.append(os.path.abspath(os.path.join("..", "src")))

# Agora você pode importar o módulo utils Ou importar funções específicas do módulo
from metrics import LogTime, eval_model
from ts_plots import plot_forecast

In [4]:
checkpoints_path = os.path.join(os.pardir, "checkpoints")
df_static = pd.read_parquet(os.path.join(checkpoints_path, "5G_df_static.parquet.gzip"))

## Running Baseline Forecast for all Uid

TO DO: Fazer que os modelos rodem para cada captura de medida, além para as médidas em driving.

Theta Forecast (All Metrics)

In [5]:
target_columns = ["RSRP", "RSRQ", "SNR", "CQI", "RSSI"]

In [6]:
static_uids = sorted(df_static.Uid.unique().tolist())

In [7]:
test_df = (
    df_static[df_static.Uid == static_uids[10]][target_columns].asfreq(freq="S").bfill()
)
ts_static = TimeSeries.from_dataframe(
    test_df,
    value_cols=target_columns,
)
ts_train, ts_actual = ts_static.split_after(0.8)

In [8]:
name = "Theta"
theta_preds = []
theta_metrics = []

with LogTime() as timer:
    for target in tqdm(target_columns):
        preds = {}

        # Need add check for seasonality and fallback rule.
        theta_model = Theta(
            theta=3, seasonality_period=60, season_mode=SeasonalityMode.ADDITIVE
        )

        tr = ts_train[target]

        vl = ts_actual[target]

        y_pred, metrics = eval_model(theta_model, tr, vl, name=name)

        # Predictions
        preds["Algorithm"] = name
        preds["Uid"] = static_uids[10]
        preds["Mobility"] = "static"
        preds["y"] = target
        preds["Timestamp"] = y_pred.time_index
        preds["y_Actual"] = vl.values()
        preds["y_Forecast"] = y_pred.values()

        # Metrics
        metrics["Uid"] = static_uids[10]
        metrics["Mobility"] = "static"
        metrics["Algorithm"] = name
        metrics["Target"] = target

        theta_preds.append(preds)
        theta_metrics.append(metrics)

100%|██████████| 5/5 [00:00<00:00, 16.03it/s]

Time Elapsed: 0 microseconds


In [9]:
pd.DataFrame(theta_preds)

,Algorithm,Uid,Mobility,y,Timestamp,y_Actual,y_Forecast
0,Theta,L2Gc6KFE,static,RSRP,"DatetimeIndex(['2019-11-28 10:00:53', '2019-11...","[[-86.0], [-86.0], [-85.0], [-85.0], [-86.0], ...","[[-85.0468969813957], [-85.09804388412402], [-..."
1,Theta,L2Gc6KFE,static,RSRQ,"DatetimeIndex(['2019-11-28 10:00:53', '2019-11...","[[-11.0], [-11.0], [-12.0], [-12.0], [-11.0], ...","[[-12.803304359325876], [-12.750196762786464],..."
2,Theta,L2Gc6KFE,static,SNR,"DatetimeIndex(['2019-11-28 10:00:53', '2019-11...","[[10.0], [10.0], [8.0], [8.0], [-3.0], [-3.0],...","[[7.845300913674976], [7.849835686819868], [8...."
3,Theta,L2Gc6KFE,static,CQI,"DatetimeIndex(['2019-11-28 10:00:53', '2019-11...","[[14.0], [14.0], [14.0], [14.0], [14.0], [14.0...","[[14.05291956666453], [14.128997277180025], [1..."
4,Theta,L2Gc6KFE,static,RSSI,"DatetimeIndex(['2019-11-28 10:00:53', '2019-11...","[[-67.0], [-67.0], [-67.0], [-67.0], [-67.0], ...","[[-67.13047775347756], [-67.33620292596444], [..."


In [10]:
theta_metrics_df = pd.DataFrame(theta_metrics).set_index(["Algorithm", "Target"])
theta_metrics_df

In [12]:
theta_metrics_df[["MAE", "MSE", "MASE"]].style.highlight_min(color="purple")

Theta Forecast (All Metrics and All Uids)

In [13]:
name = "Theta"
theta_preds = []
theta_metrics = []

with LogTime() as timer:
    for uid in tqdm(static_uids):
        test_df = df_static[df_static.Uid == uid][target_columns].asfreq(freq="S")

        ts_static = TimeSeries.from_dataframe(
            test_df,
            value_cols=target_columns,
        )

        ts_static_filled = MissingValuesFiller().transform(ts_static)

        ts_train, ts_actual = ts_static_filled.split_after(0.8)

        for target in target_columns:
            preds = {}

            # Configurar o modelo Theta
            theta_model = Theta(
                theta=3, seasonality_period=60, season_mode=SeasonalityMode.ADDITIVE
            )

            tr = ts_train[target]
            vl = ts_actual[target]

            if (
                tr.pd_dataframe().isna().all().all()
                or vl.pd_dataframe().isna().all().all()
            ):
                print(f"Skipping UID: {uid}, Target: {target} due to missing values.")
                continue

            y_pred, metrics = eval_model(theta_model, tr, vl, name=name)

            # Predictions
            preds["Algorithm"] = name
            preds["Uid"] = uid
            preds["Mobility"] = "static"
            preds["Target"] = target
            preds["Timestamp"] = y_pred.time_index
            preds["y_Actual"] = vl.values()
            preds["y_Forecast"] = y_pred.values()

            # Metrics
            metrics["Uid"] = uid
            metrics["Mobility"] = "static"
            metrics["Algorithm"] = name
            metrics["Target"] = target

            theta_preds.append(preds)
            theta_metrics.append(metrics)

  9%|▊         | 2/23 [00:00<00:06,  3.34it/s]

Skipping UID: 4u4uNEHg, Target: RSSI due to missing values.


 35%|███▍      | 8/23 [00:02<00:04,  3.29it/s]

Skipping UID: Jg5gZWT9, Target: RSSI due to missing values.


 43%|████▎     | 10/23 [00:03<00:03,  3.48it/s]

...

Skipping UID: Zk7wQePj, Target: RSSI due to missing values.


 83%|████████▎ | 19/23 [00:05<00:01,  3.58it/s]

Skipping UID: in2JGSH5, Target: RSSI due to missing values.


100%|██████████| 23/23 [00:06<00:00,  3.58it/s]

Time Elapsed: 6 seconds


In [14]:
pd.DataFrame(theta_preds)

,Algorithm,Uid,Mobility,Target,Timestamp,y_Actual,y_Forecast
0,Theta,45tHAXkk,static,RSRP,"DatetimeIndex(['2019-12-16 14:10:29', '2019-12...","[[-96.5], [-96.0], [-96.0], [-96.0], [-96.0], ...","[[-97.18986733863356], [-97.20727168376943], [..."
1,Theta,45tHAXkk,static,RSRQ,"DatetimeIndex(['2019-12-16 14:10:29', '2019-12...","[[-15.0], [-14.0], [-14.0], [-16.0], [-16.0], ...","[[-15.556421013359325], [-15.622213947960713],..."
2,Theta,45tHAXkk,static,SNR,"DatetimeIndex(['2019-12-16 14:10:29', '2019-12...","[[2.5], [4.0], [4.0], [2.0], [2.0], [2.5], [3....","[[1.6798007863201514], [2.076428275071447], [1..."
3,Theta,45tHAXkk,static,CQI,"DatetimeIndex(['2019-12-16 14:10:29', '2019-12...","[[11.0], [11.0], [11.0], [11.0], [11.0], [11.0...","[[10.971855593365499], [11.1224236900332], [11..."
4,Theta,45tHAXkk,static,RSSI,"DatetimeIndex(['2019-12-16 14:10:29', '2019-12...","[[-87.0], [-87.0], [-87.0], [-87.0], [-87.0], ...","[[-86.92635811472024], [-86.7944583134208], [-..."
...,...,...,...,...,...,...,...
104,Theta,nQK8dP5x,static,RSRP,"DatetimeIndex(['2020-02-27 20:08:10', '2020-02...","[[-99.0], [-99.0], [-99.0], [-98.0], [-98.0], ...","[[-98.92139258547077], [-98.94456411899729], [..."
105,Theta,nQK8dP5x,static,RSRQ,"DatetimeIndex(['2020-02-27 20:08:10', '2020-02...","[[-16.0], [-16.0], [-16.0], [-15.0], [-15.0], ...","[[-16.08329799788414], [-16.07265178615825], [..."
106,Theta,nQK8dP5x,static,SNR,"DatetimeIndex(['2020-02-27 20:08:10', '2020-02...","[[1.0], [2.0], [2.0], [6.0], [6.0], [6.0], [3....","[[1.1240596944078662], [1.336856328579227], [1..."
107,Theta,nQK8dP5x,static,CQI,"DatetimeIndex(['2020-02-27 20:08:10', '2020-02...","[[12.0], [11.0], [11.0], [11.0], [11.0], [11.0...","[[11.936550237113751], [12.165225409539632], [..."


In [18]:
theta_metrics_df = pd.DataFrame(theta_metrics).set_index(["Algorithm", "Target", "Uid"])
theta_metrics_df

MAE        MSE       MASE Mobility
Algorithm Target Uid                                              
Theta     RSRP   45tHAXkk  1.157352   2.022148   4.886597   static
          RSRQ   45tHAXkk  2.078219   6.661288   2.566467   static
          SNR    45tHAXkk  2.437633  10.174165   2.001009   static
          CQI    45tHAXkk  1.680816   4.607213   9.826307   static
          RSSI   45tHAXkk  1.549817   3.609320   6.324084   static
...                             ...        ...        ...      ...
          RSRP   nQK8dP5x  0.920806   1.413564   4.757797   static
          RSRQ   nQK8dP5x  1.779952   6.155635   2.415325   static
          SNR    nQK8dP5x  2.936306  14.125413   2.281650   static
          CQI    nQK8dP5x  1.822498   5.303836   2.175715   static
          RSSI   nQK8dP5x  2.972255  13.549474  17.575932   static

[109 rows x 4 columns]

In [19]:
grouped_df = theta_metrics_df.groupby(["Algorithm", "Target", "Mobility"]).mean()
grouped_df = grouped_df.reset_index()
grouped_df

,Algorithm,Target,Mobility,MAE,MSE,MASE
0,Theta,CQI,static,2.054331,7.752653,8.871453
1,Theta,RSRP,static,1.811024,9.371528,7.377745
2,Theta,RSRQ,static,1.883140,6.869668,6.513372
3,Theta,RSSI,static,2.302977,11.508120,6.887841
4,Theta,SNR,static,3.103099,18.923377,5.126549
